In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as c
import matplotlib.cm as cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
from shapely.geometry import mapping
import rioxarray
import shapefile
import csv
import os
from eofs.xarray import Eof
import cftime
import nc_time_axis


# Load shapefile and filter
shapeFile = "/glade/work/sglanvil/CCR/S2S_windowsOfOpportunity/SamarFiles/US_West.shp"
domain = gpd.read_file(shapeFile)

# Load NetCDF and grab the variable
tempFile = "/glade/campaign/cesm/development/cross-wg/S2S/sglanvil/data/tas_2m_anom_cesm2cam6v2.scenario1_s2s_data.nc"
ds = xr.open_dataset(tempFile)
temp = ds['anom']  # shape: (time, lead, lat, lon)

# Rename for rioxarray compatibility
temp = temp.rename({'lat': 'y', 'lon': 'x'})

# Assign a CRS (assume EPSG:4326 for lat/lon)
temp.rio.write_crs("EPSG:4326", inplace=True)

# Clip using geometry of selected domain
temp_clipped = temp.rio.clip(
    domain.geometry.apply(mapping),
    domain.crs,
    all_touched=False
)

# Average over space
temp_clipped_avg = temp_clipped.mean(dim=["y", "x"])
print(temp_clipped_avg)


NoDataInBounds: No data found in bounds. Data variable: anom

In [3]:
import geopandas as gpd
import xarray as xr
import numpy as np
from rasterio import features
from affine import Affine

# Load shapefile (already just CA+NV)
states = gpd.read_file("/glade/work/sglanvil/CCR/S2S_windowsOfOpportunity/SamarFiles/US_West.shp")

# Load temp data
ds = xr.open_dataset("/glade/campaign/cesm/development/cross-wg/S2S/sglanvil/data/tas_2m_anom_cesm2cam6v2.scenario1_s2s_data.nc")
temp = ds['anom']  # (time, lead, lat, lon)

lat = ds['lat'].values
lon = ds['lon'].values

# Create transform
res_lon = (lon[-1] - lon[0]) / (len(lon) - 1)
res_lat = (lat[-1] - lat[0]) / (len(lat) - 1)
transform = Affine.translation(lon[0] - res_lon / 2, lat[0] - res_lat / 2) * Affine.scale(res_lon, res_lat)

# Rasterize shape into mask
mask = features.rasterize(
    [(geom, 1) for geom in states.geometry],
    out_shape=(len(lat), len(lon)),
    transform=transform,
    fill=0,
    dtype=np.uint8
).astype(bool)

# Apply mask to temp
temp_masked = temp.where(mask)

# Average over space
temp_avg = temp_masked.mean(dim=["lat", "lon"])
print(temp_avg)


<xarray.DataArray 'anom' (time: 1150, lead: 46)> Size: 212kB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * lead     (lead) float32 184B 1.0 2.0 3.0 4.0 5.0 ... 43.0 44.0 45.0 46.0
  * time     (time) float32 5kB 7.302e+05 7.302e+05 ... 7.381e+05 7.382e+05
